In [ ]:
# importing libraries
import pandas as pd
import json
import statsmodels.api as sm


In [ ]:
# loading datasets
reg = pd.read_csv(f"./CleanedData/Income_Inequality_Regression_Ready.csv").iloc[3:]
ine = pd.read_csv(f"./CleanedData/Income_Inequality_Changes.csv").iloc[3:]


In [3]:
reg

,Destination,income_High,income_Low,income_Mid,0-16,17-34,35-54,55-66,67+,High_0-16,...,Mid_17-34,High_35-54,Low_35-54,Mid_35-54,High_55-66,Low_55-66,Mid_55-66,High_67+,Low_67+,Mid_67+
3,Ashfield_2016,1889.2002,4637.4928,5302.2870,1088.6612,2602.7868,1530.9564,477.9276,6128.6480,159.6117,...,1153.1595,241.0499,595.2654,694.6411,70.6221,192.2306,215.0749,971.7306,2411.3144,2745.6030
4,Ashford_2016,7516.5753,3168.9387,2898.9331,1247.8744,2783.2789,1795.3832,658.8096,7099.1010,721.1703,...,630.2439,988.3480,418.7651,388.2701,389.3573,151.8552,117.5971,3917.8701,1679.0623,1502.1686
5,Babergh_2016,2053.3002,1458.5515,5228.5152,832.3289,1660.8627,1170.9878,476.1372,4600.0503,183.7596,...,934.0859,244.6785,183.5648,742.7445,106.6758,70.8074,298.6540,1075.4108,770.3628,2754.2767
6,Barking and Dagenham_2016,21211.4723,1669.1483,4219.0244,3489.3517,5924.1243,3469.4847,520.9366,13695.7477,2807.0743,...,1022.3412,2774.6026,178.0990,516.7831,415.0563,22.8656,83.0147,10726.1234,847.3245,2122.2998
7,Barnet_2016,37321.1251,3320.4264,8735.8414,3612.4328,13470.8554,5977.3020,1134.6243,25182.1784,2969.5601,...,2814.3607,4928.7135,267.3233,781.2652,854.8563,84.1004,195.6676,19062.9884,1687.1882,4432.0018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,Wrexham / Wrecsam_2024,3086.5249,3236.9416,2031.9905,509.7987,2147.8474,1020.2783,340.6935,4336.8391,165.2033,...,504.3378,337.3071,426.8945,256.0767,101.2934,140.9133,98.4868,1596.9426,1690.2055,1049.6910
2876,Wychavon_2024,4536.8840,4719.6659,4852.5885,1050.9698,2827.2396,1743.5979,1011.1696,7476.1615,276.6867,...,975.1182,549.5931,592.0376,601.9672,309.7937,406.5818,294.7941,2374.2639,2546.4968,2555.4008
2877,Wyre_2024,3429.2857,6433.0227,4096.2627,1134.9555,2440.4056,1617.2333,1273.5345,7492.4422,211.1510,...,747.6944,372.1284,789.8398,455.2651,279.7882,609.0667,384.6796,1820.2942,3471.8748,2200.2732
2878,Wyre Forest_2024,2824.5723,4192.8225,1715.1503,669.5458,1914.3929,1032.5310,540.9155,4575.1599,197.8721,...,462.0661,310.9024,532.5722,189.0564,140.1060,338.8016,62.0079,1469.2563,2221.7179,884.1857


In [ ]:
#load inequality changes dataset
ine = pd.read_csv(f"./CleanedData/Income_Inequality_Changes.csv").iloc[3:]
# create unique Destination identifier by combining council and year
ine['council'] = ine['council'] + '_' + ine['year'].astype(str)
# drop unnecessary columns and rename for clarity
ine.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
# renaming council column to Destination for merging
ine.rename(columns={"council": "Destination"}, inplace=True)
# set index to Destination for easier merging
ine.set_index("Destination")
# display first 10 rows
ine.head(10)


,Destination,year,fw_index,fw_change
3,Birmingham_2019,2019,4.119175,-0.097304
4,Birmingham_2020,2020,4.155534,0.036358
5,Birmingham_2021,2021,3.884944,-0.270590
6,Birmingham_2022,2022,3.799673,-0.085271
7,Birmingham_2023,2023,3.928850,0.129177
8,Birmingham_2024,2024,3.977052,0.048202
9,Birmingham_2025,2025,3.769586,-0.207466
10,City of Edinburgh_2016,2016,4.079887,NaN
11,City of Edinburgh_2017,2017,4.124067,0.044180
12,City of Edinburgh_2018,2018,4.155805,0.031738


In [ ]:
# merging datasets on Destination
df_merged = pd.merge(ine, reg, on='Destination', how='inner')


In [ ]:
#drop NaN values in fw_change column and year column
df_merged = df_merged.dropna(subset=['fw_change'])
# drop year column
df_merged.drop(columns=["year"], axis=1, inplace=True)
#display dataframe
df_merged

,Destination,fw_index,fw_change,income_High,income_Low,income_Mid,0-16,17-34,35-54,55-66,...,Mid_17-34,High_35-54,Low_35-54,Mid_35-54,High_55-66,Low_55-66,Mid_55-66,High_67+,Low_67+,Mid_67+
0,Birmingham_2019,4.119175,-0.097304,40995.4749,21048.2824,45068.4948,6711.7002,35202.5840,9055.7750,1806.0102,...,14234.1683,3421.0072,1627.3915,4007.3763,641.7280,349.6503,814.6319,20776.8835,10679.1448,22880.1544
1,Birmingham_2020,4.155534,0.036358,34981.2892,18566.1902,45029.5323,5750.8143,32903.0310,8402.5857,1603.9052,...,13742.3823,2655.2378,1401.4317,4345.9162,416.7514,295.8621,891.2917,17608.8820,9408.4292,22899.3643
2,Birmingham_2021,3.884944,-0.270590,46946.7735,25996.2389,36474.1168,6558.3183,35501.9449,9948.8720,2052.0210,...,11337.0086,4214.8195,2208.4689,3525.5836,831.4053,494.0435,726.5722,23745.9184,13158.3745,18451.6801
3,Birmingham_2022,3.799673,-0.085271,48804.1539,29731.2534,33587.0973,6933.8597,36251.5587,10045.2554,2072.9124,...,10687.4048,4025.8817,3025.0182,2994.3555,771.6906,632.4027,668.8191,24674.3065,15102.9767,17041.6352
4,Birmingham_2023,3.928850,0.129177,52643.0460,33682.4600,28113.4585,7245.1495,36998.6307,10062.3272,2167.0268,...,9140.0619,4492.8419,3144.0248,2425.4605,904.2903,735.6352,527.1013,26651.8252,17057.0023,14257.0028
5,Birmingham_2024,3.977052,0.048202,45123.5388,41005.9314,32892.0597,7740.0729,37904.1608,11077.9969,2061.9052,...,11261.1010,3917.6441,4342.5552,2817.7976,733.9689,891.4507,436.4856,22812.7309,20857.5812,16567.0820
7,Cornwall UA_2017,3.824193,1.326775,19811.8858,13695.8739,16091.6738,3481.7901,10626.0382,6056.6640,3495.5667,...,3563.6343,2476.3546,1667.2778,1913.0316,1360.1665,1115.6279,1019.7723,10276.9490,7241.8337,8420.5918
8,Cornwall UA_2018,3.700447,-0.123746,18986.4863,10943.0526,17602.2260,3087.6460,10535.3634,5615.5005,3423.4708,...,4092.0355,2228.8627,1295.6721,2090.9657,1443.8575,860.9307,1118.6826,9903.4893,5800.5271,9165.7678
9,Cornwall UA_2019,3.864332,0.163885,18556.8584,13342.0153,17064.6494,3372.6544,10547.0542,5997.4413,3397.8468,...,4125.9830,2368.7502,1720.3440,1908.3471,1294.4287,1049.8540,1053.5641,9674.9425,7081.2383,8892.3456
10,Cornwall UA_2020,3.889493,0.025161,17843.4064,10558.1793,14682.5933,2584.5660,10045.2611,4870.0706,3089.3676,...,3649.7201,2025.5510,1198.5150,1646.0046,1284.8936,920.3831,884.0909,9281.7854,5577.5123,7635.6160


In [7]:
# Drop non-numeric columns
X = df_merged.drop(columns=['Destination', 'fw_index', 'fw_change'])
y = df_merged['fw_change']

# Add a constant term for the intercept
X = sm.add_constant(X)


In [ ]:
# Fit the OLS regression model
model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              fw_change   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     2.336
Date:                Sun, 14 Dec 2025   Prob (F-statistic):             0.0234
Time:                        17:54:24   Log-Likelihood:                 19.165
No. Observations:                  46   AIC:                            -6.331
Df Residuals:                      30   BIC:                             22.93
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0045      0.091      0.050      

In [ ]:
# summary table of OLS regression results
ols_full_df = pd.DataFrame({
    'coef': model.params,
    'std_err': model.bse,
    't': model.tvalues,
    'p_value': model.pvalues,
    'CI_lower': model.conf_int()[0],
    'CI_upper': model.conf_int()[1]
})
# display full table without truncation
pd.set_option('display.max_rows', None)
print(ols_full_df)

                 coef   std_err         t   p_value  CI_lower  CI_upper
const        0.004533  0.091257  0.049670  0.960715 -0.181838  0.190903
income_High  0.001311  0.000647  2.027888  0.051534 -0.000009  0.002631
income_Low   0.000107  0.000403  0.265439  0.792487 -0.000715  0.000929
income_Mid  -0.001810  0.000523 -3.459550  0.001644 -0.002878 -0.000741
0-16         0.000070  0.000266  0.264775  0.792994 -0.000473  0.000614
17-34       -0.000198  0.000202 -0.978737  0.335537 -0.000611  0.000215
35-54       -0.000329  0.000185 -1.778334  0.085487 -0.000707  0.000049
55-66       -0.000317  0.000444 -0.713965  0.480767 -0.001225  0.000590
67+          0.000382  0.000479  0.796533  0.431981 -0.000597  0.001361
High_0-16    0.001231  0.000519  2.374036  0.024194  0.000172  0.002290
Low_0-16    -0.000444  0.000412 -1.077020  0.290056 -0.001286  0.000398
Mid_0-16    -0.000717  0.000496 -1.444133  0.159065 -0.001731  0.000297
High_17-34   0.000939  0.000411  2.284406  0.029593  0.000099  0

This section examines if migration into an area from higher income areas is associated with yoy changes in income inequality.

In [ ]:
analysis_df = df_merged[["fw_change", "income_High"]].dropna()

y = analysis_df["fw_change"]
X = analysis_df[["income_High"]]
X = sm.add_constant(X)

model_main = sm.OLS(y, X).fit()
print(model_main.summary())

                            OLS Regression Results                            
Dep. Variable:              fw_change   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                   0.03925
Date:                Mon, 15 Dec 2025   Prob (F-statistic):              0.844
Time:                        18:45:20   Log-Likelihood:                 1.3906
No. Observations:                  46   AIC:                             1.219
Df Residuals:                      44   BIC:                             4.876
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0115      0.085     -0.136      


The coefficient on income_high is small and statistically insignificant indicating that migration from high income areas is not strongly associated with short term change in income inequality within this data set tested. 

The model captures the relationship between high income migration and income inequalty changes but by treating all migration in the same way. In reality the flows of that migration differ by age and income origin which may have effects. Therefore to explore whether the relationship varies by the type of flow of migration we will look at a more disaggregated version.

In [ ]:
cols = [
    "fw_change",
    "High_0-16", "High_17-34", "High_35-54", "High_55-66", "High_67+",
    "Mid_0-16", "Mid_17-34", "Mid_35-54", "Mid_55-66", "Mid_67+",
    "Low_0-16", "Low_17-34", "Low_35-54", "Low_55-66", "Low_67+"
]

analysis_df = df_merged[cols].dropna()

y = analysis_df["fw_change"]
X = analysis_df.drop(columns=["fw_change"])
X = sm.add_constant(X)

model_stratified = sm.OLS(y, X).fit()
print(model_stratified.summary())


                            OLS Regression Results                            
Dep. Variable:              fw_change   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     2.336
Date:                Mon, 15 Dec 2025   Prob (F-statistic):             0.0234
Time:                        18:53:35   Log-Likelihood:                 19.165
No. Observations:                  46   AIC:                            -6.331
Df Residuals:                      30   BIC:                             22.93
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0045      0.091      0.050      0.9

This model includes multiple stratified migration variables. It shows signs of multicollinearity therefore individual coefficient esitmates might be unstable. Because of this, the results are exploratory rather than causal. Despite this limitation, the model suggests that the relationship between migration and inequality differ across groups. Several high income origin age groups are positively associated with an increase in income polarisation; several mid income origin groups are negatively associated with inequality change. This indicates that the grouping of migration flows may matter more than overall volumes of migration.

In [ ]:
high_cols = [
    "High_0-16", "High_17-34", "High_35-54", "High_55-66", "High_67+"
]

analysis_df = df_merged[["fw_change"] + high_cols].dropna()

y = analysis_df["fw_change"]
X = analysis_df[high_cols]
X = sm.add_constant(X)

model_high_only = sm.OLS(y, X).fit()
print(model_high_only.summary())


                            OLS Regression Results                            
Dep. Variable:              fw_change   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                 -0.052
Method:                 Least Squares   F-statistic:                    0.5580
Date:                Mon, 15 Dec 2025   Prob (F-statistic):              0.731
Time:                        19:09:47   Log-Likelihood:                 2.9208
No. Observations:                  46   AIC:                             6.158
Df Residuals:                      40   BIC:                             17.13
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0522      0.095     -0.548      0.5

When restricting the model to high income origin groups only, it does not reveal any statstically significant associations to changes in income inequality. The coefficients are small and collectively explain little variation in fw change. The results indidcate that high income migration even when grouped by age, is not a strong predictor of short term changes in income inequality in this dataset. 